In [ ]:
import panel as pn
pn.extension('vtk')

The ``VTK`` pane renders VTK objects and vtk.js files inside a panel, making it possible to interact with complex geometries in 3D.

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

* **``axes``** (dict): A dictionary with the parameters of the axes to construct in the 3d view.
    Must contain at least ``xticker``, ``yticker`` and ``zticker``.
    - ``[x|y|z]ticker`` is a dictionary which contains:
        - ``ticks`` (array of numbers) - required. Positions in the scene coordinates
        of the coresponding axe ticks
        - ``labels`` (array of strings) - optional. Label displayed respectively to
        the `ticks` positions.

        If `labels` are not defined they are infered from the `ticks` array.
    - ``digits``: number of decimal digits when `ticks` are converted to `labels`.
    - ``fontsize``: size in pts of the ticks labels.
    - ``show_grid``: boolean. If true (default) the axes grid is visible.
    - ``grid_opacity``: float between 0-1. Defines the grid opacity.
    - ``axes_opacity``: float between 0-1. Defines the axes lines opacity.
* **``camera``** (dict): A dictionary reflecting the current state of the VTK camera
* **``enable_keybindings``** (bool): A boolean to activate/deactivate keybindings. Bound keys are:
  - s: set representation of all actors to *surface*
  - w: set representation of all actors to *wireframe*
  - v: set representation of all actors to *vertex*
  - r: center the actors and move the camera so that all actors are visible
  
  The mouse must be over the pane to work
  <br>**Warning**: These keybindings may not work as expected in a notebook context, if they interact with already bound keys
* **``orientation_widget``** (bool): A boolean to activate/deactivate the orientation widget in the 3D pane. This widget is clickable and allows to rotate the scene in one of the orthographic projections.
* **``object``** (str or object): Can be a string pointing to a local or remote file with a `.vtkjs` extension, or a `vtkRenderWindow` object  
* **``serialize_on_instantiation``** (bool): It defines when the serialization of the 3D scene occurs. If set to `True` (default) the scene object is serialized when the panel is created else when the panel is displayed to the screen. This parameter is constant, once set it can't be modified
___

The simplest way to construct a VTK pane is to give it a vtk.js file which it will serialize and embed in the plot. The ``VTK`` pane also supports the regular sizing options provided by Bokeh, including responsive sizing modes:

In [ ]:
vtk_pane = pn.pane.VTK('https://raw.githubusercontent.com/Kitware/vtk-js/master/Data/StanfordDragon.vtkjs',
                     sizing_mode='stretch_width', height=400, enable_keybindings=True, orientation_widget=True,
                     serialize_on_instantiation=True)
vtk_pane

The ``VTK`` pane can also be updated like all other pane objects by replacing the ``object``:

In [ ]:
vtk_pane.object = "https://github.com/Kitware/vtk-js-datasets/raw/master/data/vtkjs/TBarAssembly.vtkjs"

### Controls

The `VTK` pane exposes a number of options which can be changed from both Python and Javascript. Try out the effect of these parameters interactively:

In [ ]:
pn.Row(vtk_pane.controls(jslink=True), vtk_pane)

## Camera control

Once a VTK pane has been displayed it will automatically sync the camera state with the Pane object. We can read the camera state on the corresponding parameter:

```python
> vtk_pane.camera

{'position': [-21.490090356222225, 14.44963146483641, 26.581314468858984],
 'focalPoint': [0, 4.969950199127197, 0],
 'viewUp': [0.17670012087160802, 0.9635684210080306, -0.20078088883170594],
 'directionOfProjection': [0.605834463228546,
  -0.2672449261957517,
  -0.749362897791989],
 'parallelProjection': False,
 'useHorizontalViewAngle': False,
 'viewAngle': 30,
 'parallelScale': 9.180799381276024,
 'clippingRange': [26.442079567041056, 44.714416678555395],
 'thickness': 1000,
 'windowCenter': [0, 0],
 'useOffAxisProjection': False,
 'screenBottomLeft': [-0.5, -0.5, -0.5],
 'screenBottomRight': [0.5, -0.5, -0.5],
 'screenTopRight': [0.5, 0.5, -0.5],
 'freezeFocalPoint': False,
 'useScissor': False,
 'projectionMatrix': None,
 'viewMatrix': None,
 'physicalTranslation': [0, -4.969950199127197, 0],
 'physicalScale': 9.180799381276024,
 'physicalViewUp': [0, 1, 0],
 'physicalViewNorth': [0, 0, -1],
 'mtime': 2237,
 'distance': 35.47188491341284}
```

This technique also makes it possible to link the camera of two or more VTK panes together:

In [ ]:
dragon1 = pn.pane.VTK('https://raw.githubusercontent.com/Kitware/vtk-js/master/Data/StanfordDragon.vtkjs',
                      height=400, sizing_mode='stretch_width')
dragon2 = pn.pane.VTK('https://raw.githubusercontent.com/Kitware/vtk-js/master/Data/StanfordDragon.vtkjs',
                      height=400, sizing_mode='stretch_width')

dragon1.jslink(dragon2, camera='camera', bidirectional=True)

pn.Row(dragon1, dragon2)

and to modify the camera state in Python and trigger an update:

In [ ]:
if dragon1.camera:
    dragon1.camera['viewAngle'] = 50
    dragon1.param.trigger('camera')

## Rendering VTK objects

In addition to support for plotting vtkjs files the VTK pane can also render objects defined using the ``vtk`` Python library.

There are slight differences with classical code generally used. As rendering of the objects and interactions with the view are handle by the VTK panel, we don't need to make a call to the `Render` method of the `vtkRenderWindow` (which would pop up the classical VTK window) or to specify a `vtkRenderWindowInteractor`. 

In [ ]:
import vtk
from vtk.util.colors import tomato

# This creates a polygonal cylinder model with eight circumferential
# facets.
cylinder = vtk.vtkCylinderSource()
cylinder.SetResolution(8)

# The mapper is responsible for pushing the geometry into the graphics
# library. It may also do color mapping, if scalars or other
# attributes are defined.
cylinderMapper = vtk.vtkPolyDataMapper()
cylinderMapper.SetInputConnection(cylinder.GetOutputPort())

# The actor is a grouping mechanism: besides the geometry (mapper), it
# also has a property, transformation matrix, and/or texture map.
# Here we set its color and rotate it -22.5 degrees.
cylinderActor = vtk.vtkActor()
cylinderActor.SetMapper(cylinderMapper)
cylinderActor.GetProperty().SetColor(tomato)
# We must set ScalarVisibilty to 0 to use tomato Color
cylinderMapper.SetScalarVisibility(0)
cylinderActor.RotateX(30.0)
cylinderActor.RotateY(-45.0)

# Create the graphics structure. The renderer renders into the render
# window.
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)

# Add the actors to the renderer, set the background and size
ren.AddActor(cylinderActor)
ren.SetBackground(0.1, 0.2, 0.4)

geom_pane = pn.pane.VTK(renWin, width=500, height=500)

geom_pane

We can also add additional actors to the plot and then trigger an update:

In [ ]:
sphere = vtk.vtkSphereSource()

sphereMapper = vtk.vtkPolyDataMapper()
sphereMapper.SetInputConnection(sphere.GetOutputPort())

sphereActor = vtk.vtkActor()
sphereActor.SetMapper(sphereMapper)
sphereActor.GetProperty().SetColor(tomato)
sphereMapper.SetScalarVisibility(0)
sphereActor.RotateX(30.0)
sphereActor.RotateY(-45.0)
sphereActor.SetPosition(0.5, 0.5, 0.5)

ren.AddActor(sphereActor)

geom_pane.param.trigger('object')

## Construction of ColorBars for VTK objects

When working with VTK objects, a `scalar array` can be associated with `vertices` or `cells` of an `actor` `mesh`. With the help of a `look-up table`, scalar values are associated with colors at rendering time.

**Colorbars** are used to represent the association between scalars and colors. The `VTK pane` allows constructing colorbars for the `vtk actors` in the scene using the `construct_colorbars`  method. This method will return a bokeh plot containing colorbars for each actor that uses a scalar array and a lookup table.

If no colorbar is found in the scene the method return `None`.

We present an example of the use of `construct_colorbars`. Here we use `pyvista` module which simplifies the construction of VTK scenes

In [ ]:
import pyvista as pv
import numpy as np
def make_points():
    """Helper to make XYZ points"""
    theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
    z = np.linspace(-2, 2, 100)
    r = z**2 + 1
    x = r * np.sin(theta)
    y = r * np.cos(theta)
    return np.column_stack((x, y, z))

def lines_from_points(points):
    """Given an array of points, make a line set"""
    poly = pv.PolyData()
    poly.points = points
    cells = np.full((len(points)-1, 3), 2, dtype=np.int)
    cells[:, 1] = np.arange(0, len(points)-1, dtype=np.int)
    cells[:, 2] = np.arange(1, len(points), dtype=np.int)
    poly.lines = cells
    return poly

points = make_points()
line = lines_from_points(points)
line["scalars"] = np.arange(line.n_points) # By default pyvista use viridis colormap
tube = line.tube(radius=0.1) #=> the object we will represent in the scene


#pyvista plotter => it constructs a vtkRenderWindow under the attribute ren_win
pl = pv.Plotter(notebook=True)
pl.camera_position =  [(4.440892098500626e-16, -21.75168228149414, 4.258553981781006),
                       (4.440892098500626e-16, 0.8581731039809382, 0),
                       (0, 0.1850949078798294, 0.982720673084259)]

pl.add_mesh(tube, smooth_shading=True)
pan = pn.panel(pl.ren_win, sizing_mode='stretch_width', orientation_widget=True)
pn.Row(pn.Column(pan, pan.construct_colorbars()), pn.pane.Str(type(pl.ren_win), width=500))

## Adding axes to the VTK scene

Using the `axes` parameter, it is possible to add an axes into the 3d view.

In [ ]:
axes = dict(
    origin = [-5, 5, -2],
    xticker = {'ticks': np.linspace(-5,5,5)},
    yticker = {'ticks': np.linspace(-5,5,5)},
    zticker = {'ticks': np.linspace(-2,2,5), 'labels': [''] + [str(int(item)) for item in np.linspace(-2,2,5)[1:]]},
    fontsize = 12,
    digits = 1,
    grid_opacity = 0.5,
    show_grid=True
)
pan.axes = axes